In [1]:
import boolexpr as bx
import itertools

import cProfile
import signal
from contextlib import contextmanager


In [2]:
n = 21
vertices = list(range(n))
edges = list(itertools.combinations(vertices, 2))
ctx = bx.Context()
variables = { **{ (i,j,0) : ctx.get_var(str((i,j,0))) for i,j in edges} ,
             **{ (i,j,1) : ctx.get_var(str((i,j,1))) for i,j in edges} ,  
             **{ (i,j,2) : ctx.get_var(str((i,j,2))) for i,j in edges} }

In [6]:

# exactly one color is true
conjuncts_colors = ( (variables[(i,j,0)] | variables[(i,j,1)] | variables[(i,j,2)]) 
                    & (~variables[(i,j,0)] | ~variables[(i,j,1)]) 
                    & (~variables[(i,j,0)] | ~variables[(i,j,2)]) 
                    & (~variables[(i,j,1)] | ~variables[(i,j,2)])  
                    for i,j in edges )

phi_0 = bx.and_s(*conjuncts_colors)


# R = {(0,1)} | {(2,x) for x in range(3,10)} \
#   | {(3,x) for x in range(4,10)} | {(4,x) for x in range(5,10)} | {(5,x) for x in range(6,10)} \
#   | {(0,10),(1,10)}  # red K_3

# R = {(0,1)} | {(2,x) for x in range(3,10)} \
#   | {(3,x) for x in range(4,10)} | {(4,x) for x in range(5,10)} | {(5,x) for x in range(6,10)} \
#   | {(0,10),(1,10), (0,11),(1,11),(10,11)}  # red K_4

# R = {(0,1)} | {(2,x) for x in range(3,10)} \
#   | {(3,x) for x in range(4,10)} | {(4,x) for x in range(5,10)} | {(5,x) for x in range(6,10)} \
#   | {(0,10),(1,10), (0,11),(1,11),(10,11),(0,12),(1,12),(10,12),(11,12)}  # red K_5

R = {(0,1)} | {(2,x) for x in range(3,10)} \
  | {(3,x) for x in range(4,10)} | {(4,x) for x in range(5,10)} | {(5,x) for x in range(6,10)} \
  | {(0,10),(1,10), (0,11),(1,11),(10,11),(0,12),(1,12),(10,12),(11,12),(0,13),(1,13),(10,13),(11,13),(12,13)}  # red K_6

Bl = {(0,2),(1,2),(0,3),(1,4),(1,6),(0,7)}
Bd = {(1,3),(0,4),(0,5),(1,5),(1,8),(0,9)}

conjuncts_red_subgraph = (variables[(i,j,0)] for i,j in R)

conjuncts_lblue_subgraph = (variables[(i,j,1)] for i,j in Bl)

conjuncts_dblue_subgraph = (variables[(i,j,2)] for i,j in Bd)


#color the existing edges by what we know
phi_1 = bx.and_s( bx.and_s(*conjuncts_red_subgraph), bx.and_s(*conjuncts_lblue_subgraph),
                bx.and_s(*conjuncts_dblue_subgraph) )


def disjunction(i,j,c1,c2):
    # returns a disjunction that says edge ij has need c1-c2 met
    return bx.or_s( *( variables[(min(i,k), max(i,k), c1)] 
                               & variables[(min(j,k), max(j,k), c2)] for k in set(range(n)) - {i,j} ) ) 

# each edge must have its "needs" met

#def conjuncts_needs(c1, c2)

conjuncts_needs_red = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in itertools.product({0,1,2}, {0,1,2}) ))  
                       for i,j in R)

# all possible needs for blue edges
edge_needs_bl = {(0,0),(0,1),(0,2),(1,0),(2,0)}


#every red edge must have its "needs" met
phi_2 = bx.and_s(*conjuncts_needs_red)


conjuncts_no_blue_triangles = ( variables[(i,j,0)] | variables[(i,k,0)] | variables[(j,k,0)] 
                                 for i,j,k in itertools.combinations(vertices,3))

#forbid all blue triangle 
phi_3 = bx.and_s(*conjuncts_no_blue_triangles)


#every edge must have its "needs" met
conjuncts_needs_Bl = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_bl ))  
                       for i,j in Bl)


#every light blue edge must have its "needs" met
phi_4 = bx.and_s(*conjuncts_needs_Bl)

conjuncts_needs_Bd = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_bl ))  
                       for i,j in Bd)


#every dark blue edge must have its "needs" met
phi_5 = bx.and_s(*conjuncts_needs_Bd)



# every edge not colored must have its "needs" met
edge_unknown = set(set(edges) - set(R) - set(Bl) - set(Bd))

def needs_r(i,j):
    # returns a disjunction that says edge ij is red and has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in itertools.product({0,1,2}, {0,1,2}) ))  
def needs_b(i,j):
    # returns a disjunction that says edge ij is blue has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_bl )) 


conjuncts_needs_unknown =  (  (variables[(i,j,0)] & needs_r(i,j) ) |  
                                (variables[(i,j,1)] & needs_b(i,j) ) |  
                                (variables[(i,j,2)] & needs_b(i,j) ) 
                              for i,j in edge_unknown)

phi_6 = bx.and_s(*conjuncts_needs_unknown)




# phi = bx.and_(phi_0, phi_1, phi_2)
phi = bx.and_s(phi_0, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6)

phi_simpl = phi.simplify()
phi_tseytin = phi.tseytin(ctx)

# with open('tseytin.txt', 'w') as f:
#     print('phi_tseytin:', phi_tseytin, file=f)  # Python 3.x

#print(phi_simpl)
#print(phi)
#phi_tseytin.sat()

#phi_simpl.sat()

#list(phi_simpl.iter_sat())


class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


try:
    with time_limit(1000):
        cProfile.run('phi_tseytin.sat()')
except TimeoutException as e:
    print("Timed out!")


phi_tseytin.sat()

         10 function calls in 119.083 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  119.079  119.079 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 wrap.py:123(__init__)
        1    0.000    0.000    0.000    0.000 wrap.py:126(__del__)
        2    0.000    0.000    0.000    0.000 wrap.py:129(sat)
        1    0.000    0.000    0.000    0.000 wrap.py:134(point)
        1    0.000    0.000    0.000    0.000 wrap.py:141(t)
        1  119.079  119.079  119.079  119.079 wrap.py:611(sat)
        1    0.003    0.003  119.083  119.083 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




(False, None)

In [4]:
# either the edge is red, in which case all 9 needs must be satisfied
# or the edge is blue, where use edge_needs_bl, only 5/9 can be satisfied and the other four are forbidden

In [5]:
# all known edges satisfying their needs
# trial 1 , 40 min still no result
# trial 2 , using phi_tseytin, started 14:55, end? still going as of 18:35
# trial 3 , using phi_tseytin, started 18:41
# trial 4, phi_simpl simplified expression run start 20:30, 11:50 the next day, still running....